In [1]:
from collections import defaultdict, Counter

import collections
import numpy as np
import operator

In [2]:
DATA_PATH = './data.csv'

In [3]:
def load_dataset():
    data = np.genfromtxt(DATA_PATH, dtype=str, delimiter=',')
    print(data.shape)
    test  = []
    train = []
    for clazz in np.unique(data[:,1]):
        data_class = data[data[:,1] == clazz]
        count = int(data_class.shape[0]*.8)
    
        train.append(data_class[:count])
        test.append(data_class[count:])
        
    return np.concatenate(train), np.concatenate(test)

In [4]:
train_samples, test_samples = load_dataset()
print(train_samples.shape, test_samples.shape)

(1118, 2)
(894, 2) (224, 2)


In [5]:
class NaiveBayes:
    def __init__(self):
        self.container = dict()
        self.prior = defaultdict(int)
    
        self.unique_words = 0
    
    def initialize_dict_labels(self, labels):
        container = dict()
        for label in np.unique(labels):
            container[label] = defaultdict(int)
        
        return container
            
    def count_priors(self):
        for label in self.data[:,1]:
            self.prior[label] += 1

        shape = self.data.shape[0]
        for key, value in self.prior.items():
            self.prior[key] = value/shape
    
    def __fit_tfidf(self):
        self.idf_container = self.initialize_dict_labels(self.data[:,1])

        for clazz, values in self.container.items():
            for word, freq in values.items():
                for doc in self.data[:, 0]:
                    if word in doc:
                        self.idf_container[clazz][word] += 1
                
    def fit(self, data, tfidf=False):        
        self.data = np.asarray(data)
        self.container = self.initialize_dict_labels(self.data[:,-1])
        
        all_words = []
        for clazz, values in self.container.items():
            class_words = []
            for sample in self.data[self.data[:,-1] == clazz]:
                words = sample[0].lower().split()
                class_words.append(words)
                all_words.append(words)

            class_words = np.concatenate(class_words)                                                                 
            self.container[clazz] = Counter(class_words)

        if tfidf:
            self.__fit_tfidf()
            
        all_words = np.concatenate(all_words)
        unique_words = Counter(all_words)

        self.unique_words = unique_words.keys()
        self.count_priors()
        
    def __predict_tfidf(self, document, logarifmic=True):
        words = document[0].lower().split(' ')
        class_predictions = dict()

        for clazz, values in self.container.items():
            total_count = sum(values.values())
            container = []
            
            for word in words:
                if word == '':
                    pass
                else: 
                    P = (1 + values[word])/(total_count + len(self.unique_words))*\
                         np.log(total_count/(self.idf_container[clazz][word] + 1))
                    container.append(P)
            
            if logarifmic:
                class_predictions[clazz] = np.log(self.prior[clazz])
                for value in container:
                    class_predictions[clazz] += np.log(value)

            else:
                class_predictions[clazz] = self.prior[clazz]
                for value in container:
                    class_predictions[clazz] *= value
                    
        return class_predictions

    
    def _predict(self, document, logarifmic=True):
        words = document[0].lower().split(' ')
        class_predictions = dict()

        for clazz, values in self.container.items():
            total_count = sum(values.values())
            container = []
            
            for word in words:
                if word == '':
                    pass
                else: 
                    if word in values.keys():
                        value = values[word]
                    else:
                        value = 0

                    P = (value + 1)/(total_count + len(self.unique_words))
                    container.append(P)
            
            if logarifmic:
                class_predictions[clazz] = np.log(self.prior[clazz])
                for value in container:
                    class_predictions[clazz] += np.log(value)

            else:
                class_predictions[clazz] = self.prior[clazz]
                for value in container:
                    class_predictions[clazz] *= value
                    
        return class_predictions

    def predict(self, samples, tfidf=False):
        container = []
        for i in samples:
            result = self.__predict_tfidf(i) if tfidf else self._predict(i)
            maximum = max(result.items(), key=operator.itemgetter(1))[0]
            container.append(maximum == i[1])
        return np.asarray(container).mean()

In [6]:
nb = NaiveBayes()
nb.fit(train_samples)

In [7]:
len(nb.unique_words)

26456

In [8]:
result = nb.predict(train_samples)
result

0.9910514541387024

In [9]:
result = nb.predict(test_samples)
result

0.9642857142857143

In [10]:
# Train data
data = [["Chinese Beijing Chinese","0"],
            ["Chinese Chinese Shanghai","0"], 
            ["Chinese Macao","0"],
            ["Tokyo Japan Chinese","1"]]

# Fit model
s = NaiveBayes()
s.fit(data)

In [11]:
# Mpdel predict
result = s._predict(["Chinese Chinese Chinese Tokyo Japan"], logarifmic=False)
result

{'0': 0.00030121377997263036, '1': 0.00013548070246744226}

In [ ]:
# Must return[ ('Chinese Chinese Chinese Tokyo Japan', '0')]
# pobability {'1': 0.00013548070246744226, '0': 0.00030121377997263036}
# or log     {'1': -7.906681345001262, '0': -7.10769031284391}